**Do not forget your TODO items**

----------------------

## Imports

In [29]:
from itertools import chain, tee
from collections import deque
import math
import functools

import numpy as np

## Load the cities

In [30]:
from us_city import load_us_cities
cities, norm_cities = load_us_cities()

## Initialize hyperparameters

In [31]:
DIM = 2 # dimension of the points
N = cities.shape[0]  # number of cities
M = int(2.5 * N)  # number of intermediate points between the cities

KAPPA_START = 0.2  # starting value of kappa
GAMMA = 1.05  # kappa damping factor

BETA = 10 # inverse temperature

NUM_ITERS = 10000 # number of iterations of the algorithm
WORST_DISTANCE_EPSILON = 0.01

PERTURBATION_RADIUS = 1 # perturbation radius for generating the intermediate points

# TODO: Delete the two parameters below
# Elastic Net parameters
ELASTIC_ALPHA = 0.2
ELASTIC_BETA = 2.0

## 1. Implement the elastic net TSP by minimizing the following objective function.

$E_{TSP}(P, Y) = \sum_{i = 1}^{N}{\sum_{a = 1}^M{P_{ia}||x_i - y_a||^2}} + \frac{\kappa}{2}\sum_{a = 1}^M{||y_a - y_{a \oplus 1}||_2^2}$

In [32]:
class ElasticNet(object):
    
    def __init__(
        self,
        init_intermediate_cities,
        solve,
        update_weights,
        update_kappa,
        update_intermediate_cities,
        get_D
    ):
        # Take the functions as parameters to be able to split the
        # method definitions into separate cells.
        ElasticNet.init_intermediate_cities = init_intermediate_cities
        ElasticNet.solve = solve
        ElasticNet.update_weights = update_weights
        ElasticNet.update_kappa = update_kappa
        ElasticNet.update_intermediate_cities = update_intermediate_cities
        ElasticNet.get_D = get_D

        # Initialize kappa to the starting value
        self.kappa = KAPPA_START
        # Set cities to the normalized cities
        self.cities = norm_cities
        # Generate intermediate cities around the centroid
        self.init_intermediate_cities(strategy="centroid")
        # A queue to hold the last two worst distances (for the stopping criterion)
        self.worst_dists = deque(maxlen=2)


    @functools.cached_property
    def L(self):
        def L_ij(i, j, M):
            if i == j:
                return 2
            if i == (j + 1) % M:
                return 1
            if j == (i + 1) % M:
                return 1
            return 0
        return np.fromfunction(np.vectorize(L_ij), (M, M), M=M, dtype=int)

In [33]:
def init_intermediate_cities(self, strategy="centroid", **kwargs):
    if strategy == "centroid":
        self.intermediate_cities = np.tile(
            np.mean(self.cities, axis=0),
            M
        ).reshape((M, DIM))
        # Create perturbation vectors of magnitude radius
        perturbation_vectors = np.random.random((M, DIM))
        perturbation_vectors /= np.linalg.norm(perturbation_vectors, axis=1, ord=2, keepdims=True)
        perturbation_vectors *= PERTURBATION_RADIUS
        self.intermediate_cities += perturbation_vectors
    else:
        raise RuntimeError(f"Unknown strategy: {strategy}")

In [34]:
def solve(self):
    for i in range(NUM_ITERS):
        self.update_weights()
        self.update_kappa()
        self.update_intermediate_cities()
        if i % 100 == 0:
            print(f"Iteration {i}, Worst distance: {self.worst_dist}")
        if self.worst_dist < WORST_DISTANCE_EPSILON:
            break
        if len(self.worst_dists) == 2:
            if self.worst_dists[1] - self.worst_dists[0] == 0:
                break

In [35]:
def update_kappa(self):
    self.kappa = max(self.kappa / GAMMA, 0.01)

In [36]:
def update_weights(self):
    self.diff = self.cities[:,np.newaxis] - self.intermediate_cities
    self.dist_squared = np.sum(self.diff ** 2, axis=-1)

    self.worst_dist = np.max(np.min(np.sqrt(self.dist_squared), axis=1))
    self.worst_dists.append(self.worst_dist)

    self.weights = np.exp(-self.dist_squared / (2 * (self.kappa ** 2)))
    self.weights /= self.weights.sum(axis=1)[:,np.newaxis]

In [37]:
def get_D(self):
    def d_ij(i, j):            
        if i == j:
            return np.sum(self.weights[:, int(j)])
        return 0
    return np.fromfunction(np.vectorize(d_ij), (M, M))

In [38]:
def update_intermediate_cities(self):
    self.intermediate_cities = np.linalg.inv((self.kappa * self.L + self.get_D())) @ self.weights.T @ self.cities

In [39]:
net = ElasticNet(
    init_intermediate_cities=init_intermediate_cities,
    solve=solve,
    update_weights=update_weights,
    update_kappa=update_kappa,
    update_intermediate_cities=update_intermediate_cities,
    get_D=get_D
)
net.solve()

Iteration 0, Worst distance: 1.5200382619389061


In [40]:
net.worst_dist

0.07155421587255416

In [41]:
net.weights

array([[0.00000000e+000, 0.00000000e+000, 2.93159123e-202, ...,
        0.00000000e+000, 1.29984687e-053, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       ...,
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 0.00000000e+000, 0.00000000e+000],
       [0.00000000e+000, 0.00000000e+000, 0.00000000e+000, ...,
        0.00000000e+000, 5.86081923e-196, 0.00000000e+000]])

In [42]:
net.intermediate_cities

array([[ 0.00825436,  0.59527569],
       [-0.19668904, -0.64470498],
       [ 0.38512372,  0.69413428],
       [-0.3850973 , -0.69412682],
       [ 0.38507088,  0.69411936],
       [-0.19434921, -0.64403618],
       [ 0.00362755,  0.593953  ],
       [-0.01557843, -0.34229082],
       [ 0.0275293 ,  0.09062865],
       [-0.02765355, -0.08945623],
       [ 0.02777781,  0.08828381],
       [-0.02790361, -0.24002302],
       [ 0.02802941,  0.39176224],
       [-0.04315334, -0.63400036],
       [ 0.05827727,  0.87623848],
       [-0.34557251, -0.93744672],
       [ 0.63286775,  0.99865496],
       [-0.31804885, -0.92535966],
       [ 0.00322996,  0.85206435],
       [-0.01140664, -0.61935004],
       [ 0.01958332,  0.38663573],
       [ 0.30326224, -0.00816605],
       [-0.52614106, -0.01434142],
       [ 0.74901987,  0.03684889],
       [-0.87388596, -0.08570869],
       [ 0.99875205,  0.13456848],
       [-0.99882287, -0.22564252],
       [ 0.9988937 ,  0.31671656],
       [-0.88833441,

In [43]:
def decode_solution(dist2):
    """Return the permutation associated to the elastic."""

    neuron_city_pair = []

    for _ in range(dist2.shape[0]):
        # Find the city 
        city = dist2.min(axis=1).argmin()
        neuron = dist2[city].argmin()

        dist2[city] = np.inf
        dist2[:, neuron] = np.inf

        neuron_city_pair.append((neuron, city))

    neuron_city_pair.sort(key=lambda x: x[0])
    return [x[1] for x in neuron_city_pair]


def compute_solution(permutation):
    pass    


def _pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)


def compute_solution(permutation, original_cities):
    edges = list(_pairwise(chain(permutation, [permutation[0]])))

    length = sum([distance(original_cities[src_dst[0]],
                           original_cities[src_dst[1]]) for src_dst in edges])

    return (length, edges)


def distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return int(math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) + 0.5)


In [44]:
permutation = decode_solution(net.dist_squared)
permutation

[8, 5, 11, 19, 13, 3, 1, 4, 20, 18, 15, 12, 10, 6, 2, 7, 9, 17, 21, 14, 0, 16]

In [45]:
compute_solution(permutation, cities)

(540,
 [(8, 5),
  (5, 11),
  (11, 19),
  (19, 13),
  (13, 3),
  (3, 1),
  (1, 4),
  (4, 20),
  (20, 18),
  (18, 15),
  (15, 12),
  (12, 10),
  (10, 6),
  (6, 2),
  (2, 7),
  (7, 9),
  (9, 17),
  (17, 21),
  (21, 14),
  (14, 0),
  (0, 16),
  (16, 8)])